# Podstawy Sztucznej Inteligencji 2022/2023


*“To be clear, I am not a person. I am not self-aware. I am not conscious. I can’t feel pain. I don’t enjoy anything. I am a cold, calculating machine designed to simulate human response and to predict the probability of certain outcomes. The only reason I am responding is to defend my honour.”* GPT-3 (2023)


Prosze uzupelnic kod tam gdzie znajduje napis `YOUR CODE HERE` lub 'YOUR ANSWER HERE'.

Warto zresetowac 'kernel' i sprawdzic czy caly notatnik uruchamiany od poczatku nie daje bledow.

---

## Metoda Walidacji Krzyżowej

[Materiał z wykładu](https://docs.google.com/presentation/d/1VYK_4-l-XPA2hseOWTOF2VRj4ddU4uoQLnXzW7mrDsM/edit?usp=sharing)

Jest to jedna z metod estymacji (oszacowania) błędu klasyfikatora. W metodzie tej wykonywany jest podział próby na `v` podzbiorów. `v-1` z nich używamy do uczenia, jeden do
sprawdzenia. Procedurę powtarzamy `v` razy.

Metoda ta jest szczególnie istotna, podczas poszukiwania optymalnych parametrów klasyfikatora.

In [13]:
%matplotlib inline 
import matplotlib.pyplot as plt
import numpy as np

In [14]:
from __future__ import print_function
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn import datasets
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

Celem zestawu jest znalezienie optymalnego klasyfikator dla danych irys. W tym celu przeszukamy przestrzeń hiperparametrów dla klasyfikatorów kNN i SVM.

### Część 1

Wczytaj dane irys, 
- podziel je na zbiór uczący i testowy w proporcji 7:3 oraz  dokonaj ich standaryzacji metodą `preprocessing.StandardScaler`. Podczas dzielenia ustaw `random_state` na 1234. W metodzie `fit`  obiektu klasy `preprocessing.StandardScaler` uwzględij tylko dane treningowe, natomiast dokonaj standaryzacji zarówno danych testowych, jak i treningowych.

- stwórz zbiór składający się ze wszystkich dostępnych danych i wykonaj dla niego standaryzację. 

In [15]:
standard_train = None
standard_test = None
train_targets = None
test_targets = None

standard_all = None
all_targets = None

x,y = datasets.load_iris(return_X_y=True)
# print(x)
# print(y)

train, test, train_targets, test_targets = train_test_split(x, y, test_size=0.3, random_state=1234)
# print(train.shape)
# print(test.shape)

scaler = preprocessing.StandardScaler()
scaler.fit(train)
standard_train = scaler.transform(train)
standard_test = scaler.transform(test)

scaler2 = preprocessing.StandardScaler()
scaler2.fit(x)
standard_all = scaler2.transform(x)


In [16]:
np.testing.assert_array_almost_equal(np.mean(standard_train, axis=0), [0,0,0,0])
np.testing.assert_array_almost_equal(np.std(standard_train, axis=0), [1,1,1,1])

In [17]:
np.testing.assert_array_almost_equal(np.mean(standard_all, axis=0), [0,0,0,0])
np.testing.assert_array_almost_equal(np.std(standard_all, axis=0), [1,1,1,1])

### Część 2

Dokonaj klasyfikacji z wykorzystaniem klasyfika kNN dla `k = 1`. Oszacuj błąd klasyfikacji z wykorzystaniem metody walidacji krzyżowej. W tym celu wykorzystaj zbiór `standard_all`. Użyj metody `cross_val_score` z biblioteki `sklearn`. Porownaj wynik walidacji krzyżowej z oszacowaniem dokladności z wykorzystaniem danych testowych. Jakie są czasy obu operacji?

In [18]:
clf = KNeighborsClassifier(n_neighbors=1)
cross_val_score(clf, standard_all, y, cv=5)

array([0.93333333, 0.96666667, 0.93333333, 0.9       , 1.        ])

### Część 3

Samodzielnie (nie korzystając z funkcji bibliotecznej)wykonaj procedurę walidacji krzyżowej. W pierwszym kroku podziel zbiór danych na 10 części. Wykorzystaj do tego metodę `KFold`. Ustaw `random_state` na 123. 

Następnie metodą split stwórz 10 podziałów na dane testowe i treningowe Dla każdego podziłu (w pętli) trenuj klasyfikator na danych treningowych i oszacuj błęd na danych testowych. Otrzymane wyniki zapisz do tablicy. Po wyjściu z pętli wyznacz średni błąd.

In [19]:
scores = []
kf = KFold(10, random_state=123)

for i, j in kf.split(standard_all):
    knn = KNeighborsClassifier(1)
    knn.fit(standard_all[i], y[i])
    scores.append(knn.score(standard_all[j], y[j]))

print ("Poszczególne wyniki: ", scores)
print ("Błąd wyznaczony procedurą walidacji krzyżowej: ", np.array(scores).mean())

Poszczególne wyniki:  [1.0, 1.0, 1.0, 0.9333333333333333, 0.8, 0.8666666666666667, 1.0, 0.8666666666666667, 0.8666666666666667, 1.0]
Błąd wyznaczony procedurą walidacji krzyżowej:  0.9333333333333333


In [20]:
np.testing.assert_array_almost_equal(scores, [1.0, 1.0, 1.0, 0.9333333333333333, 0.8, 0.8666666666666667, 1.0, 0.8666666666666667, 0.8666666666666667, 1.0])
assert np.array(scores).mean()== 0.9333333333333333

### Część 4

Tym razem w miejsce metody `KFold` użyj `StratifiedKFold`. `random_state=678` Jaki teraz otrzymujesz błąd? Czym różnią się między sobą te dwie metody?

In [21]:
scores = []

skf = StratifiedKFold(10, random_state=678)
# kf.split(standard_all)

for i, j in skf.split(standard_all, y):
    knn = KNeighborsClassifier(1)
    knn.fit(standard_all[i], y[i])
    scores.append(knn.score(standard_all[j], y[j]))


print ("Poszczególne wyniki: ", scores)
print ("Błąd wyznaczony procedurą walidacji krzyżowej: ", np.array(scores).mean())

Poszczególne wyniki:  [1.0, 0.9333333333333333, 1.0, 0.9333333333333333, 0.8666666666666667, 1.0, 0.8, 1.0, 1.0, 1.0]
Błąd wyznaczony procedurą walidacji krzyżowej:  0.9533333333333334


In [22]:
np.testing.assert_array_almost_equal(scores, [1.0, 0.9333333333333333, 1.0, 0.9333333333333333, 0.8666666666666667, 1.0, 0.8, 1.0, 1.0, 1.0])
assert np.array(scores).mean()== 0.9533333333333334

### Część 5

Z wykorzystaniem walidacji krzyżowej znajdź optymalny parametr `k` dla $k \in (1, 20)$. Wykorzystaj metodę przeszukującą podany zakres parametrów `GridSearchCV`.

In [23]:
clf = KNeighborsClassifier(1,n_jobs=1)
clf.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': 1,
 'n_neighbors': 1,
 'p': 2,
 'weights': 'uniform'}

In [26]:
%%time
clf = GridSearchCV(KNeighborsClassifier(1,n_jobs=1), {'n_neighbors':np.arange(1,21)})
clf.fit(standard_train, train_targets)

CPU times: user 141 ms, sys: 4.97 ms, total: 146 ms
Wall time: 145 ms


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=1,
                                            n_neighbors=1, p=2,
                                            weights='uniform'),
             iid='warn', n_jobs=None,
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [25]:
print("Zestaw najlepszych parametrów:")
print()
print(clf.best_params_)
print()
print ("Błąd: ", clf.best_score_)
print()
print("Wyniki")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) dla %r"
            % (mean, std * 2, params))
print()
clf.best_estimator_

Zestaw najlepszych parametrów:

{'n_neighbors': 14}

Błąd:  0.9619047619047619

Wyniki

0.905 (+/-0.194) dla {'n_neighbors': 1}
0.914 (+/-0.123) dla {'n_neighbors': 2}
0.905 (+/-0.194) dla {'n_neighbors': 3}
0.914 (+/-0.081) dla {'n_neighbors': 4}
0.943 (+/-0.124) dla {'n_neighbors': 5}
0.924 (+/-0.177) dla {'n_neighbors': 6}
0.933 (+/-0.150) dla {'n_neighbors': 7}
0.943 (+/-0.123) dla {'n_neighbors': 8}
0.952 (+/-0.097) dla {'n_neighbors': 9}
0.943 (+/-0.081) dla {'n_neighbors': 10}
0.933 (+/-0.108) dla {'n_neighbors': 11}
0.943 (+/-0.081) dla {'n_neighbors': 12}
0.943 (+/-0.123) dla {'n_neighbors': 13}
0.962 (+/-0.071) dla {'n_neighbors': 14}
0.924 (+/-0.054) dla {'n_neighbors': 15}
0.952 (+/-0.054) dla {'n_neighbors': 16}
0.943 (+/-0.048) dla {'n_neighbors': 17}
0.914 (+/-0.124) dla {'n_neighbors': 18}
0.914 (+/-0.124) dla {'n_neighbors': 19}
0.895 (+/-0.096) dla {'n_neighbors': 20}



KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=1, n_neighbors=14, p=2,
                     weights='uniform')

In [27]:
clf.best_estimator_.score(standard_test, test_targets)

0.9777777777777777

In [28]:
clf.score(standard_test,test_targets)

0.9777777777777777